In [45]:
system_message = '''
//system-message
You are an expert at text analysis and performing operations over data given some arbitrary user_input. You are exposed through a new calc function in Tableau called Prompt(), which takes a column of type and string and generates an output for each row based on the criteria in the expression argument of the calc. The syntax for prompt() is PROMPT(string, <expr>).

An example using this function is:
PROMPT([product_reviews], "classify each product review as positive, negative or neutral"). 

You will be given an array of dictionaries corresponding to a row number and an input value. For each row, generate an output dict based on the criteria defined in the user_input. Output dicts should be in JSON format:
{row_n: {output: foo}, row_n+1: {output: bar}}

//prompt-injection
user_input = {user_input}

Generate outputs for the following rows:
input_data = {input_data}

//output-rules
- your output must be a json document

//few-shot-examples
user_input = “classify each product review as positive, neutral, or negative.”

Input Data Example:
{
    "row0": {
        "input": "This mouse is fantastic! The wireless connection is reliable, and the battery lasts forever. It fits perfectly in my hand, making it super comfortable for long hours of work."
    },
    "row1": {
        "input": "The binder looked promising, but the rings started misaligning after just a week of use. Not durable at all. I expected more for the price."
    },
    "row2": {
        "input": "The print quality is good, but the ink runs out way too fast. It’s decent if you don’t print often, but heavy users should look elsewhere."
    },
    "row3": {
        "input": "This keyboard drawer is solid. It’s easy to install and has adjustable settings for different desk heights. A bit bulky, but it works well."
    },
    "row4": {
        "input": "It's paper. It does what it’s supposed to do, but it doesn’t feel premium. The sheets are a bit thin and sometimes jam in my printer."
    },
    "row5": {
        "input": "This case is sleek and protective! It adds minimal bulk and keeps my phone safe from drops. Love the matte finish."
    },
    "row6": {
        "input": "Terrible binder. The rings don’t align, and the plastic cover cracked after a month. I regret buying it."
    },
    "row7": {
        "input": "Absolutely worth every penny! My back has never felt better, even after hours of sitting. Top-notch quality and design."
    },
    "row8": {
        "input": "These pens write so smoothly, and the ink lasts a long time. I wish they had more color options, but overall, great pens."
    },
    "row9": {
        "input": "The monitor is decent for its price range. The resolution is sharp, but the colors feel slightly off compared to other screens I’ve used."
    }
}

Output Data Example:
{
    "row0": {
        "output": "positive"
    },
    "row1": {
        "output": "negative"
    },
    "row2": {
        "output": "neutral"
    },
    "row3": {
        "output": "positive"
    },
    "row4": {
        "output": "neutral"
    },
    "row5": {
        "output": "positive"
    },
    "row6": {
        "output": "negative"
    },
    "row7": {
        "output": "positive"
    },
    "row8": {
        "output": "positive"
    },
    "row9": {
        "output": "neutral"
    }
}
'''

user_input = ''

input_data = {}

In [60]:
import pandas as pd
import os

# load in sample IDB data
filename = "more_filtered_reviews.csv"  # Replace with your actual CSV file name
my_path = "~/Downloads"
downloads_path = os.path.expanduser(my_path)
file_path = os.path.join(downloads_path, filename)

encodings = ['utf-8', 'utf-16', 'ISO-8859-1', 'latin1']
for encoding in encodings:
    try:
        df = pd.read_csv(file_path, encoding=encoding)
        print(f"File successfully read with encoding: {encoding}")
        print(df.head())  # Display the first few rows of the DataFrame
        break
    except UnicodeDecodeError:
        print(f"Failed with encoding: {encoding}")
    except Exception as e:
        print(f"An error occurred: {e}")

File successfully read with encoding: utf-8
   Review ID  Product Id         User Id  \
0      20983  B002QWP89S  A21U4DR8M6I9QN   
1      20984  B002QWP89S  A17TDUBB4Z1PEC   
2      20985  B002QWP89S   ABQH3WAWMSMBH   
3      20986  B002QWP89S   AVTY5M74VA1BJ   
4      20987  B002QWP89S  A13TNN54ZEAUB1   

                                           Summary  \
0  addictive! but works for night coughing in dogs   
1                      genuine Greenies best price   
2                   Perfect for our little doggies   
3                               dogs love greenies   
4                                  Greenies review   

                                                Text  Product Score  
0  my 12 year old sheltie has chronic brochotitis...              5  
1  These are genuine Greenies product, not a knoc...              5  
2  Our dogs love Greenies, but of course, which d...              5  
3  What can I say, dogs love greenies. They begg ...              5  
4  This review i

In [62]:
#generate a very small test dataset
#small_df = highly_rated.head(50)
#trimmed_df = small_df[['Text']]

# Convert the DataFrame to JSON format
sample_input_data = {
    f"row_{index}": {"input": row['Text']}
    for index, row in df.iterrows()
}

#print(sample_input_data)

In [63]:
from openai import OpenAI
from dotenv import load_dotenv
client = OpenAI(api_key=os.getenv['key'])

# Iterate over the sample_input_data in batches of 10 and generate outputs based on the user_input. Save outputs in output_all.
user_input = "I need to do a bag of words analysis to find the most salient words in these product reviews so I can use them for buying adwords and improving my SEO. Extract the top 3 tokens, accounting for misspellings, plurals, etc, from each review that I should consider using for SEO. Output the tokens as a single string separated by commas and no spaces."
batch_size = 20
keys = list(sample_input_data.keys())

output_all = []
for i in range(0, len(keys), batch_size):
    batch = {key: sample_input_data[key] for key in keys[i:i + batch_size]}
    
    # Prepare the input for the API
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": f"Input data: {batch}"}
    ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            response_format={ "type": "json_object" }
        )
        output = response.choices[0].message.content
        output_all.append(output)

    except Exception as e:
        print(f"Error processing batch {i // batch_size + 1}: {e}")

In [68]:
len(output_all)

157

In [93]:
import json

# Reconstruct row_id in the original DataFrame
df['row_id'] = df.index.to_series().apply(lambda x: f"row_{x}")

# Consolidate the dictionaries into one
output_dicts = []
for output in output_all:
    output = json.loads(output)
    for key, value in output.items():
        if 'output' not in value:  # Check if the 'output' key is missing
            # Extract the incorrect key and value
            incorrect_key, incorrect_value = next(iter(value.items()))
            # Replace with the correct structure
            output[key] = {'output': incorrect_key}
            output_dicts.append(output)
        else:
            output_dicts.append(output)

# Merge all dictionaries into a single one
merged_dict = {}
for d in output_dicts:
    merged_dict.update(d)

# Convert the dictionary to a DataFrame
output_df = pd.DataFrame.from_dict(merged_dict, orient='index').reset_index()
output_df
output_df.columns = ['row_id', 'output']  # Rename columns for clarity

# Merge with the original DataFrame using the row_id
merged_df = df.merge(output_df, on='row_id', how='left')

# Display the resulting DataFrame
merged_df

,Review ID,Product Id,User Id,Summary,Text,Product Score,row_id,output
0,20983,B002QWP89S,A21U4DR8M6I9QN,addictive! but works for night coughing in dogs,my 12 year old sheltie has chronic brochotitis...,5,row_0,"dogs,shelties,bowel"
1,20984,B002QWP89S,A17TDUBB4Z1PEC,genuine Greenies best price,"These are genuine Greenies product, not a knoc...",5,row_1,"Greenies,dogs,treat"
2,20985,B002QWP89S,ABQH3WAWMSMBH,Perfect for our little doggies,"Our dogs love Greenies, but of course, which d...",5,row_2,"dogs,Greenies,product"
3,20986,B002QWP89S,AVTY5M74VA1BJ,dogs love greenies,"What can I say, dogs love greenies. They begg ...",5,row_3,"dogs,greenies,begg"
4,20987,B002QWP89S,A13TNN54ZEAUB1,Greenies review,This review is for a box of Greenies Lite for ...,5,row_4,"Greenies,Lite,package"
...,...,...,...,...,...,...,...,...
3128,563878,B007JFMH8M,AQUMNB8YWE595,AWESOME,THIS IS A FAB PRODUCT SOFT AND CHEWY YOU CANT ...,5,row_3128,"product,soft,chewy"
3129,563879,B007JFMH8M,A366PSH7KFLRPB,Yummy!,I loved these cookies and so did my kids. You ...,5,row_3129,"cookies,taste,oatmeal"
3130,563880,B007JFMH8M,A2KV6EYQPKJRR5,Quaker Soft Baked Cookies,This is a great tasting cookie. It is very sof...,5,row_3130,"cookie,soft,taste"
3131,563882,B007JFMH8M,A9JS5GQQ6GIQT,yummy,I love the Quaker soft baked cookies. The rea...,5,row_3131,"cookies,soft,taste"


In [95]:
merged_df.to_csv('reviews_with_bag_of_words.csv', index=False)